In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf
import pandas as pd
import xlwings as xw
from datetime import datetime

In [23]:
book = xw.Book('Richspread.xlsx') 
sht = book.sheets['New_Opt']
ticker = sht.range('B2').value # Current stock price
K = sht.range('B3').value  # Strike price
expiration_date = sht.range('B4').value  # Expiration date of the option
Option_type = sht.range('B5').value  # 'call' or 'put'


print(f'Ticker: {ticker}')
print(f'Strike Price: {K}')
print(f'Expiration Date: {expiration_date}')
print(f'Option Type: {Option_type}')

e_date_c = sht.range('B4').value  # Replace with your desired expiration date string
e_date_p = sht.range('B4').value  # Replace with your desired expiration date string

Ticker: AAPL
Strike Price: 200.0
Expiration Date: 2025-07-18 00:00:00
Option Type: put


In [24]:
# Use the existing Ticker object
ticker_obj = yf.Ticker(ticker)

# Print the available expiration dates for options
print(ticker_obj.options)

# Convert e_date_c and e_date_p to string format 'YYYY-MM-DD'
e_date_c_str = e_date_c.strftime('%Y-%m-%d')
e_date_p_str = e_date_p.strftime('%Y-%m-%d')

# Load current call options data
if e_date_c_str not in ticker_obj.options:
    raise ValueError(f"Expiration date {e_date_c_str} not found in available options: {ticker_obj.options}")
calls = ticker_obj.option_chain(e_date_c_str).calls

# Load current put options data
if e_date_p_str not in ticker_obj.options:
    raise ValueError(f"Expiration date {e_date_p_str} not found in available options: {ticker_obj.options}")
puts = ticker_obj.option_chain(e_date_p_str).puts

# Print the expiration date and the first few rows of the call options data
print(f"Expiration date for calls: {e_date_c_str}")
print(calls.head())

# Print the expiration date and the first few rows of the put options data
print(f"Expiration date for puts: {e_date_p_str}")
print(puts.head())

('2025-07-18', '2025-07-25', '2025-08-01', '2025-08-08', '2025-08-15', '2025-08-22', '2025-08-29', '2025-09-19', '2025-10-17', '2025-11-21', '2025-12-19', '2026-01-16', '2026-02-20', '2026-03-20', '2026-05-15', '2026-06-18', '2026-09-18', '2026-12-18', '2027-01-15', '2027-06-17', '2027-12-17')
Expiration date for calls: 2025-07-18
        contractSymbol             lastTradeDate  strike  lastPrice  bid  ask  \
0  AAPL250718C00090000 2025-07-11 15:38:33+00:00    90.0     120.77  0.0  0.0   
1  AAPL250718C00095000 2025-07-11 15:38:33+00:00    95.0     115.75  0.0  0.0   
2  AAPL250718C00100000 2025-07-10 19:44:53+00:00   100.0     113.00  0.0  0.0   
3  AAPL250718C00105000 2025-07-11 18:09:40+00:00   105.0     106.55  0.0  0.0   
4  AAPL250718C00110000 2025-07-11 18:09:40+00:00   110.0     101.54  0.0  0.0   

   change  percentChange  volume  openInterest  impliedVolatility  inTheMoney  \
0     0.0            0.0     2.0           405            0.00001        True   
1     0.0         

In [25]:
sht.range('A20:Z1000').clear_contents()

# Export calls and puts DataFrames to Excel using xlwings, starting at A20
sht.range('A20').value = 'Calls'
sht.range('A21').options(index=False).value = calls

# Find the next empty row after the calls table to write puts, add 2 for a blank row
next_row = sht.range('A21').expand('table').rows.count + 32  # +40 leaves one blank row between tables
sht.range(f'A{next_row}').value = 'Puts'
sht.range(f'A{next_row+1}').options(index=False).value = puts

print("Exported calls and puts data to Excel sheet starting at A20 with a blank row in between.")

Exported calls and puts data to Excel sheet starting at A20 with a blank row in between.


In [26]:
# Ticker symbol for asset
# ticker = yf.Ticker(ticker)  # Already a Ticker object, do not re-initialize

# Get data for call options
expiration_date = pd.to_datetime(e_date_c, utc=True)
now_day = expiration_date - 5 * pd.tseries.offsets.BDay()
now_text = str(now_day).split(' ')[0]
if Option_type.lower() == 'call':
    option = calls[calls['strike'] == K].iloc[0]
elif Option_type.lower() == 'put':
    option = puts[puts['strike'] == K].iloc[0]
else:
    raise ValueError("Option_type must be 'call' or 'put'")

# Get the current price of the basic asset
hist = ticker_obj.history()
# Convert now_text to the same timezone as hist.index
now_dt = pd.to_datetime(now_text).tz_localize(hist.index.tz)
available_dates = hist.index[hist.index <= now_dt]
if len(available_dates) == 0:
    raise ValueError(f"No available trading data on or before {now_text}")
last_trading_day = available_dates[-1]
current_price = hist['Close'].loc[last_trading_day]

implied_volatility = option['impliedVolatility']  # Convert percentage to decimal

# Calculate date for option expiration in years scale
time_to_expiration = (expiration_date - now_day).days / 365.0



print('\n', option)


 contractSymbol             AAPL250718P00200000
lastTradeDate        2025-07-11 19:59:52+00:00
strike                                   200.0
lastPrice                                 0.21
bid                                        0.0
ask                                        0.0
change                                     0.0
percentChange                              0.0
volume                                  8806.0
openInterest                             37100
impliedVolatility                     0.062509
inTheMoney                               False
contractSize                           REGULAR
currency                                   USD
Name: 27, dtype: object


In [27]:
# Convert all values in the Series to native Python types for Excel compatibility
option_native = option.apply(lambda x: x.item() if hasattr(x, "item") else x)

# Export the selected option Series to cell E2 as a column
sht.range('E2').options(index=True, header=False).value = option_native

print("Exported selected option data to cell E2.")

Exported selected option data to cell E2.
